<a href="https://colab.research.google.com/github/s3nh/s3nhutils/blob/main/Download_Youtube_WAV_and_Splitting_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!rm -rf  /content/drive/MyDrive/Colab

In [1]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#@title Input Form

Mode = "Splitting" #@param ["Separate", "Splitting"]
dataset = "Youtube" #@param ["Youtube", "Drive"]
url = "https://www.youtube.com/watch?v=z-AqkCSJUy8" #@param {type:"string"}
drive_path = "/content/drive/MyDrive/dataset/" #@param {type:"string"}
AUDIO_NAME = "bb" #@param {type:"string"}


In [3]:
#@title 1. Install Library for Youtube WAV Download
if dataset == "Drive":
  print("Dataset is set to Drive. Skipping this section")
elif dataset == "Youtube":
  !pip install yt_dlp
  !pip install ffmpeg
  !mkdir youtubeaudio

mkdir: cannot create directory ‘youtubeaudio’: File exists


In [4]:
#@title Download Youtube WAV
from __future__ import unicode_literals

if dataset == "Drive":
  print("Dataset is set to Drive. Skipping this section")
elif dataset == "Youtube":
  import yt_dlp
  import ffmpeg
  import sys


  ydl_opts = {
      'format': 'bestaudio/best',
  #    'outtmpl': 'output.%(ext)s',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'wav',
      }],
      "outtmpl": f'youtubeaudio/{AUDIO_NAME}',  # this is where you can edit how you'd like the filenames to be formatted
  }
  def download_from_url(url):
      ydl.download([url])
      # stream = ffmpeg.input('output.m4a')
      # stream = ffmpeg.output(stream, 'output.wav')


  with yt_dlp.YoutubeDL(ydl_opts) as ydl:

        download_from_url(url)

[youtube] Extracting URL: https://www.youtube.com/watch?v=z-AqkCSJUy8
[youtube] z-AqkCSJUy8: Downloading webpage
[youtube] z-AqkCSJUy8: Downloading ios player API JSON
[youtube] z-AqkCSJUy8: Downloading android player API JSON
[youtube] z-AqkCSJUy8: Downloading m3u8 information
[info] z-AqkCSJUy8: Downloading 1 format(s): 140
[download] Destination: youtubeaudio/bb
[download] 100% of   11.66MiB in 00:00:00 at 49.76MiB/s  
[FixupM4a] Correcting container of "youtubeaudio/bb"
[ExtractAudio] Destination: youtubeaudio/bb.wav
Deleting original file youtubeaudio/bb (pass -k to keep)


In [5]:
#@title 2. Install Demucs for Separating Audio
!python3 -m pip install -U demucs

In [6]:
#@title Separate Vocal and Instrument/Noise using Demucs
import subprocess
AUDIO_INPUT = f"/content/youtubeaudio/{AUDIO_NAME}.wav"

if dataset == "Drive":
  command = f"demucs --two-stems=vocals {drive_path}"
elif dataset == "Youtube":
  command = f"demucs --two-stems=vocals {AUDIO_INPUT}"

result = subprocess.run(command.split(), stdout=subprocess.PIPE)
print(result.stdout.decode())

Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /content/separated/htdemucs
Separating track /content/youtubeaudio/bb.wav



In [8]:
!mkdir -p /content/drive/MyDrive/audio/{AUDIO_NAME}
!cp -r /content/separated/htdemucs/{AUDIO_NAME}/* /content/drive/MyDrive/audio/{AUDIO_NAME}
if dataset == "Youtube":
  !cp -r /content/youtubeaudio/{AUDIO_NAME}.wav /content/drive/MyDrive/audio/{AUDIO_NAME}


In [9]:
#@title 3. Split The Audio into Smaller Duration Before Training
if Mode == "Separate":
    print("Mode is set to Separate. Skipping this section")
elif Mode ==  "Splitting":
  !pip install numpy
  !pip install librosa
  !pip install soundfile
  !mkdir -p dataset/{AUDIO_NAME}

In [10]:
#@title
import numpy as np
import librosa
import soundfile


# This function is obtained from librosa.
def get_rms(
    y,
    *,
    frame_length=2048,
    hop_length=512,
    pad_mode="constant",
):
    padding = (int(frame_length // 2), int(frame_length // 2))
    y = np.pad(y, padding, mode=pad_mode)

    axis = -1
    # put our new within-frame axis at the end for now
    out_strides = y.strides + tuple([y.strides[axis]])
    # Reduce the shape on the framing axis
    x_shape_trimmed = list(y.shape)
    x_shape_trimmed[axis] -= frame_length - 1
    out_shape = tuple(x_shape_trimmed) + tuple([frame_length])
    xw = np.lib.stride_tricks.as_strided(
        y, shape=out_shape, strides=out_strides
    )
    if axis < 0:
        target_axis = axis - 1
    else:
        target_axis = axis + 1
    xw = np.moveaxis(xw, -1, target_axis)
    # Downsample along the target axis
    slices = [slice(None)] * xw.ndim
    slices[axis] = slice(0, None, hop_length)
    x = xw[tuple(slices)]

    # Calculate power
    power = np.mean(np.abs(x) ** 2, axis=-2, keepdims=True)

    return np.sqrt(power)


class Slicer:
    def __init__(self,
                 sr: int,
                 threshold: float = -40.,
                 min_length: int = 5000,
                 min_interval: int = 300,
                 hop_size: int = 20,
                 max_sil_kept: int = 5000):
        if not min_length >= min_interval >= hop_size:
            raise ValueError('The following condition must be satisfied: min_length >= min_interval >= hop_size')
        if not max_sil_kept >= hop_size:
            raise ValueError('The following condition must be satisfied: max_sil_kept >= hop_size')
        min_interval = sr * min_interval / 1000
        self.threshold = 10 ** (threshold / 20.)
        self.hop_size = round(sr * hop_size / 1000)
        self.win_size = min(round(min_interval), 4 * self.hop_size)
        self.min_length = round(sr * min_length / 1000 / self.hop_size)
        self.min_interval = round(min_interval / self.hop_size)
        self.max_sil_kept = round(sr * max_sil_kept / 1000 / self.hop_size)

    def _apply_slice(self, waveform, begin, end):
        if len(waveform.shape) > 1:
            return waveform[:, begin * self.hop_size: min(waveform.shape[1], end * self.hop_size)]
        else:
            return waveform[begin * self.hop_size: min(waveform.shape[0], end * self.hop_size)]

    def slice(self, waveform):
        if len(waveform.shape) > 1:
            samples = waveform.mean(axis=0)
        else:
            samples = waveform
        if samples.shape[0] <= self.min_length:
            return [waveform]
        rms_list = get_rms(y=samples, frame_length=self.win_size, hop_length=self.hop_size).squeeze(0)
        sil_tags = []
        silence_start = None
        clip_start = 0
        for i, rms in enumerate(rms_list):
            # Keep looping while frame is silent.
            if rms < self.threshold:
                # Record start of silent frames.
                if silence_start is None:
                    silence_start = i
                continue
            # Keep looping while frame is not silent and silence start has not been recorded.
            if silence_start is None:
                continue
            # Clear recorded silence start if interval is not enough or clip is too short
            is_leading_silence = silence_start == 0 and i > self.max_sil_kept
            need_slice_middle = i - silence_start >= self.min_interval and i - clip_start >= self.min_length
            if not is_leading_silence and not need_slice_middle:
                silence_start = None
                continue
            # Need slicing. Record the range of silent frames to be removed.
            if i - silence_start <= self.max_sil_kept:
                pos = rms_list[silence_start: i + 1].argmin() + silence_start
                if silence_start == 0:
                    sil_tags.append((0, pos))
                else:
                    sil_tags.append((pos, pos))
                clip_start = pos
            elif i - silence_start <= self.max_sil_kept * 2:
                pos = rms_list[i - self.max_sil_kept: silence_start + self.max_sil_kept + 1].argmin()
                pos += i - self.max_sil_kept
                pos_l = rms_list[silence_start: silence_start + self.max_sil_kept + 1].argmin() + silence_start
                pos_r = rms_list[i - self.max_sil_kept: i + 1].argmin() + i - self.max_sil_kept
                if silence_start == 0:
                    sil_tags.append((0, pos_r))
                    clip_start = pos_r
                else:
                    sil_tags.append((min(pos_l, pos), max(pos_r, pos)))
                    clip_start = max(pos_r, pos)
            else:
                pos_l = rms_list[silence_start: silence_start + self.max_sil_kept + 1].argmin() + silence_start
                pos_r = rms_list[i - self.max_sil_kept: i + 1].argmin() + i - self.max_sil_kept
                if silence_start == 0:
                    sil_tags.append((0, pos_r))
                else:
                    sil_tags.append((pos_l, pos_r))
                clip_start = pos_r
            silence_start = None
        # Deal with trailing silence.
        total_frames = rms_list.shape[0]
        if silence_start is not None and total_frames - silence_start >= self.min_interval:
            silence_end = min(total_frames, silence_start + self.max_sil_kept)
            pos = rms_list[silence_start: silence_end + 1].argmin() + silence_start
            sil_tags.append((pos, total_frames + 1))
        # Apply and return slices.
        if len(sil_tags) == 0:
            return [waveform]
        else:
            chunks = []
            if sil_tags[0][0] > 0:
                chunks.append(self._apply_slice(waveform, 0, sil_tags[0][0]))
            for i in range(len(sil_tags) - 1):
                chunks.append(self._apply_slice(waveform, sil_tags[i][1], sil_tags[i + 1][0]))
            if sil_tags[-1][1] < total_frames:
                chunks.append(self._apply_slice(waveform, sil_tags[-1][1], total_frames))
            return chunks

if Mode == "Separate":
    print("Mode is set to Separate. Skipping this section")

elif Mode ==  "Splitting":
  audio, sr = librosa.load(f'/content/separated/htdemucs/{AUDIO_NAME}/vocals.wav', sr=None, mono=False)  # Load an audio file with librosa.
  slicer = Slicer(
      sr=sr,
      threshold=-40,
      min_length=5000,
      min_interval=200,
      hop_size=10,
      max_sil_kept=500
  )
  chunks = slicer.slice(audio)
  for i, chunk in enumerate(chunks):
      if len(chunk.shape) > 1:
          chunk = chunk.T  # Swap axes if the audio is stereo.
      soundfile.write(f'/content/dataset/{AUDIO_NAME}/split_{i}.wav', chunk, sr)  # Save sliced audio files with soundfile.

In [11]:
if Mode == "Separate":
    print("Mode is set to Separate. Skipping this section")
elif Mode ==  "Splitting":
  !mkdir -p /content/drive/MyDrive/dataset/{AUDIO_NAME}
  !cp -r /content/dataset/{AUDIO_NAME}/* /content/drive/MyDrive/dataset/{AUDIO_NAME}

In [12]:
#@title Input Form
mode = "training" #@param ["inference", "training"]
#@markdown fill modelname if training/resume training
MODELNAME = "bb"  #@param {type:"string"}



In [13]:
#@title Cloning Github

!git clone https://github.com/RVC-Project/Retrieval-based-Voice-Conversion-WebUI.git
!cp -r /content/drive/MyDrive/RVC/weights/* /content/Retrieval-based-Voice-Conversion-WebUI/weights
!mkdir -p /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
!cp -r /content/drive/MyDrive/RVC/{MODELNAME}/* /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}
%cd /content/Retrieval-based-Voice-Conversion-WebUI
!mkdir -p pretrained uvr5_weights

fatal: destination path 'Retrieval-based-Voice-Conversion-WebUI' already exists and is not an empty directory.
cp: cannot stat '/content/drive/MyDrive/RVC/bb/*': No such file or directory
/content/Retrieval-based-Voice-Conversion-WebUI


In [14]:
!pip install pyworld==0.3.2
!pip install -r requirements.txt

Ignoring onnxruntime: markers 'sys_platform == "darwin"' don't match your environment


In [15]:
#@title Install aria2
!apt -y install -qq aria2

aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [16]:
#RVC V2
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0D48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0D48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G32k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G40k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/pretrained_v2/f0G48k.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2 -o f0G48k.pth


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
cbaf9d|OK  |       0B/s|/content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2/D32k.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
0a397e|OK  |       0B/s|/content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2/D40k.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
e26f3a|OK  |       0B/s|/content/Retrieval-based-Voice-Conversion-WebUI/pretrained_v2/D48k.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
f5b260|OK  |       0B/s|/content/Retrieval-based-Voice-Conversio

In [17]:
#@title Download Pretrained Model for Audio Splitting
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2-人声vocals+非人声instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5-主旋律人声vocals+其他instrumentals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/5_HP-Karaoke-UVR.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o 5_HP-Karaoke-UVR.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP2_all_vocals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP2_all_vocals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP3_all_vocals.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP3_all_vocals.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/HP5_only_main_vocal.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o HP5_only_main_vocal.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoAggressive.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o VR-DeEchoAggressive.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoDeReverb.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o VR-DeEchoDeReverb.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/uvr5_weights/VR-DeEchoNormal.pth -d /content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights -o VR-DeEchoNormal.pth



Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
589874|OK  |       0B/s|/content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights/HP2-人声vocals+非人声instrumentals.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
500530|OK  |       0B/s|/content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights/HP5-主旋律人声vocals+其他instrumentals.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
976c91|OK  |       0B/s|/content/Retrieval-based-Voice-Conversion-WebUI/uvr5_weights/5_HP-Karaoke-UVR.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
c9c

In [18]:
#@title Download hubert_base
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/hubert_base.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/lj1995/VoiceConversionWebUI/resolve/main/rmvpe.pt -d /content/Retrieval-based-Voice-Conversion-WebUI -o rmvpe.pt


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
00fa34|OK  |       0B/s|/content/Retrieval-based-Voice-Conversion-WebUI/hubert_base.pt

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
c5262f|OK  |       0B/s|/content/Retrieval-based-Voice-Conversion-WebUI/rmvpe.pt

Status Legend:
(OK):download completed.


In [ ]:
#@title Run Web
%cd /content/Retrieval-based-Voice-Conversion-WebUI
# %load_ext tensorboard
# %tensorboard --logdir /content/Retrieval-based-Voice-Conversion-WebUI/logs
!python3 infer-web.py --colab --pycmd python3

/content/Retrieval-based-Voice-Conversion-WebUI
2023-08-21 07:47:48.251436: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-21 07:47:49.659576: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Found GPU Tesla T4
Use Language: en_US
Running on local URL:  http://127.0.0.1:7860

Setting up a public link... we have recently upgraded the way public links are generated. If you encounter any problems, please report the issue and downgrade to gradio version 3.13.0
.
Running on public URL: https://8010c63e-9225-46a7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
loading weights/bb.pth
gin_channels: 

In [ ]:
#@title Copy Training Result to Drive
if mode == "training":
  !mkdir -p /content/drive/MyDrive/RVC/{MODELNAME}
  !mkdir -p /content/drive/MyDrive/RVC/weights

  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/G_*.pth /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/D_*.pth /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/added_*.index /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/trained*.index /content/drive/MyDrive/RVC/{MODELNAME}
  !cp /content/Retrieval-based-Voice-Conversion-WebUI/logs/{MODELNAME}/total_*.npy /content/drive/MyDrive/RVC/{MODELNAME}

  !cp /content/Retrieval-based-Voice-Conversion-WebUI/weights/{MODELNAME}.pth /content/drive/MyDrive/RVC/weights/{MODELNAME}.pth
elif mode == "inference":
  print("Mode set to inference. Skipping this section")